In [ ]:
# Ioana Alexandra Mititean
# Reducing Imaging Data
# ASTR 480

In [1]:
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import glob

In [2]:
bias_list = [line.rstrip('\n') for line in open("input_bias.txt")] 

In [52]:
# Combining bias frames and printing the mean values for all frames and the combined frame

data_stack = []
for file in bias_list:
    file_data = fits.getdata(file).astype(np.int32)
    data_stack.append(file_data)
    #print("Mean value for " + file + ": " + str(np.mean(file_data)))
    
meanBias = np.mean(data_stack, axis=0)
header = fits.getheader(bias_list[0])
header['HISTORY'] = 'Mean combined'
#fits.writeto('master_bias_mean.fits', meanBias, header) 

master_data = fits.getdata("master_bias_mean.fits").astype(np.int32)
#print("Mean value for master bias: " + str(np.mean(master_data)))

In [8]:
# Putting all flat and science frames into a single list
image_list = glob.glob('flat*.fits')
wolf_list = glob.glob('wolf*.fits')
for file in wolf_list:
    image_list.append(file)
   

In [9]:
# Removing the bias from all flat field images and wolf1346 frames
datafilesout = [line.rstrip("\n") for line in open("output_no_bias.txt")]
list_length = len(image_list)
for i in range(0, list_length): 
    data, header = fits.getdata(image_list[i], header = True)
    data_out = data - meanBias
    header['HISTORY'] = 'Bias subtracted'
    
    # I tried to fix the program refusal to write to the output files (because of some 
    # cards in the image file headers that read 'NAN'); forcing the program to ignore
    # the non-standard format seemed to work,and I find it okay to use because I don't 
    #need the 'CD*_*' cards for this assignment. The 'fixed' line is written below 
    # (commented out)
    
    #fits.writeto(datafilesout[i], data_out, header, output_verify = 'ignore')

In [50]:
# Printing out the filter names for each flat image file (optional - helped me see at a glance which
# types of filters each flat was used with)
flat_list = image_list
for flat in flat_list:
    header = fits.getheader(flat)
    #print("Filter name for " + flat + ": " + header['FILTER'])


In [38]:
# Reading in lists for each type of filter
washmflatfiles = [line.rstrip("\n") for line in open("washm_flats_input.txt")]
jcrcflatfiles = [line.rstrip("\n") for line in open("jc-rc_flats_input.txt")]

# Scale each image in each filter list by its median
washmflat_stack = []
for file in washmflatfiles: 
    data,header = fits.getdata(file,header=True)
    data = data / np.median(data)
    washmflat_stack.append(data)
    
washmflat = np.median(washmflat_stack,axis=0)
m = np.mean(washmflat)
washmflat = washmflat/m
    
header["HISTORY"] = "Combined and normalized flat field"
fits.writeto("washmflat.fits", washmflat, header, output_verify = 'ignore') 
    
jcrcflat_stack = []
for file in jcrcflatfiles: 
    data,header = fits.getdata(file,header=True)
    data = data / np.median(data)
    jcrcflat_stack.append(data)

jcrcflat = np.median(jcrcflat_stack,axis=0)
m = np.mean(jcrcflat)
jcrcflat = jcrcflat/m

header["HISTORY"] = "Combined and normalized flat field"
fits.writeto("jcrcflat.fits", jcrcflat, header, output_verify = 'ignore') 

 [astropy.io.fits.verify]


In [49]:
# Flat field data

#Read in the lists of bias-subtracted data for the 'Wash M' filter
washmdatain = [line.rstrip("\n") for line in open("washm_data_bs.txt")]
washmdataout = [line.rstrip("\n") for line in open("washm_data_frm.txt")]

# Divide each data image of the 'Wash M' filter by the normalized flat field of the 'Wash M' filter
n=len(washmdatain)
for i in range(0,n): 
    data,header = fits.getdata(washmdatain[i],header=True)
    dataout = data / washmflat
    header["HISTORY"] = "Flat Fielded"
    #fits.writeto(washmdataout[i], dataout, header, output_verify = 'ignore')
    
#Read in the lists of bias-subtracted data for the 'J-C Rc' filter
jcrcdatain = [line.rstrip("\n") for line in open("jc-rc_data_bs.txt")]
jcrcdataout = [line.rstrip("\n") for line in open("jc-rc_data_frm.txt")]

# Divide each data image of the 'J-C Rc' filter by the normalized flat field of the 'J-C Rc' filter
n=len(jcrcdatain)
for i in range(0,n): 
    data,header = fits.getdata(jcrcdatain[i],header=True)
    dataout = data / jcrcflat
    header["HISTORY"] = "Flat Fielded"
    #fits.writeto(jcrcdataout[i], dataout, header, output_verify = 'ignore')